[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepmipt/RDLS_NLP_2022/blob/main/lectures/04.Creating_Dialog_Systems/seminar/notebooks/11.telegram.ipynb)

In [ ]:
# !pip install df_engine df_runner df_db_connector df_telegram_connector

<img src="https://raw.githubusercontent.com/kudep/RDLS_images/4d1061ad112b1ba4592da0b37bb4ec5b4b032167/dfe_examples/infrastructure.svg" >

In [ ]:
# imports
from df_engine.core.keywords import TRANSITIONS, RESPONSE
import df_engine.conditions as cnd
# a dialog script
script = {
    "greeting_flow": {
        "start_node": {  # This is an initial node, it doesn't need an `RESPONSE`
            RESPONSE: "",
            TRANSITIONS: {"node1": cnd.exact_match("Hi")},  # If "Hi" == request of user then we make the transition
        },
        "node1": {
            RESPONSE: "Hi, how are you?",  # When the agent goes to node1, we return "Hi, how are you?"
            TRANSITIONS: {"node2": cnd.regexp(r".*(good|fine|great).*")},
        },
        "node2": {
            RESPONSE: "Good. What do you want to talk about?",
            TRANSITIONS: {"node3": cnd.regexp(r"(music[.!]{0,1}|.*about music[.!]{0,1})")},
        },
        "node3": {
            RESPONSE: "Sorry, I can not talk about music now.",
            TRANSITIONS: {"node4": cnd.exact_match("Ok, goodbye.")},
        },
        "node4": {RESPONSE: "bye", TRANSITIONS: {"node1": cnd.regexp(r".*(restart|start|start again).*")}},
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: "Ooops",
            TRANSITIONS: {"node1": cnd.true()},
        },
    }
}

In [ ]:
import os

from df_runner import ScriptRunner
from df_telegram_connector.connector import TelegramConnector
from df_telegram_connector.request_provider import PollingRequestProvider

bot = TelegramConnector(os.getenv("BOT_TOKEN", "SOMETOKEN"))
db = dict()

provider = PollingRequestProvider(bot=bot)
runner = ScriptRunner(
    script=script,
    start_label=("greeting_flow", "start_node"),
    fallback_label=("greeting_flow", "fallback_node"),
    db=db,
    request_provider=provider,
)

In [ ]:
runner.start()